In [17]:
import random

f = open("dataset_865386_4.txt","r")

k, t, N = [int(x) for x in next(f).split()]
Dna = [str(x) for x in next(f).split()]

#k,t,N = 8, 5, 100
#Dna = ["CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA", "GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG", "TAGTACCGAGACCGAAAGAAGTATACAGGCGT", "TAGATCAAGTTTCAGGTGCACGTCGGTGAACC", "AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

PSEUDO_OFFSET = 4
RANDOM_ITERATIONS = 100

def GibbsSampleXTimes(Dna, k, t, N, iterations):
    BestMotifs = GibbsSampler(Dna, k, t, N)
    BestScore = Score(BestMotifs)
    
    for i in range(iterations):
        motifs = GibbsSampler(Dna, k, t, N)
        score = Score(BestMotifs)
        
        if Score(motifs) < BestScore:
            BestMotifs = motifs
            BestScore = Score(BestMotifs)
    
    return BestMotifs

def GibbsSampler(Dna, k, t, N):
    BestMotifs = []
    for i in range(len(Dna)):
        r = random.randint(0, len(Dna[i]) - k)
        BestMotifs.append(Dna[i][r:r + k])
    
    for j in range(N):
        i = int(random.random() * t)
        Motifs = []
        for l in range(len(BestMotifs)):
            if i != l:
                Motifs.append(BestMotifs[l])
        
        profile = FindProfile(Motifs, k)
        Motifs.insert(i, ProfileRandomMostProbableKmer(Dna[i], k, profile))
        
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs = Motifs
            
    return BestMotifs    
    
        
def FindProfile(motifs, k):
    Profile = []
    for i in range(k):
        Profile.append({"A":1.0,"C":1.0,"G":1.0,"T":1.0})
    for motif in motifs:
        for nucleotide in range(len(motif)):
            Profile[nucleotide][motif[nucleotide]] += 1.0
    for i in range(k):
        for key in Profile[i]:
            Profile[i][key] /= (len(motifs) + PSEUDO_OFFSET)
            
    return Profile
    
def ProfileRandomMostProbableKmer(Text, k, Profile):
    kmers = []
    probabilities = []
    
    for i in range(len(Text) - k + 1):
        kmer = Text[i:i + k]
        prob = 1
        for n in range(len(kmer)):
            prob = prob * Profile[n][kmer[n]]
            
        probabilities.append(prob)
        kmers.append(kmer)
        
    return kmers[Random(probabilities)]

def Score(motifs):
    Profile = []
    k = len(motifs[0])
    for j in range(k):
        Profile.append({"A":0.0,"C":0.0,"G":0.0,"T":0.0})
    for motif in motifs:
        for nucleotide in range(len(motif)):
            Profile[nucleotide][motif[nucleotide]] += 1.0
    
    score = 0
    for i in range(k):
        max_key = "A"
        max_val = float('-inf')
        for key in Profile[i]:
            if Profile[i][key] > max_val:
                max_val = Profile[i][key]
                max_key = key
        
        for key in Profile[i]:
            if key != max_key:
                score += Profile[i][key]
                
    return score

def Random(p):
    r = random.random()
    C = sum(p)

    s = 0.0
    for i in range(len(p)):
        s += (p[i] / C)
        if s >= r:
            return i
    
out = ""
for motif in GibbsSampleXTimes(Dna, k, t, N, RANDOM_ITERATIONS):
    out = out + motif + " "
print(out)
    
f.close()

ATGTGGATGATCCGG AGGGCCAATATCGGG AGGACTTATATCGGG AGGAGGAATATCTAC AGGAGGAATATACAG AGGAGGTTGATCGGG ACCCGGAATATCGGG AGGCTAAATATCGGG AGGAGGAATTGGGGG AGGAGCTTTATCGGG AGTCCGAATATCGGG AGGATTCATATCGGG CGGAGGAATATCGAT CACAGGAATATCGGG AGGAGGCCCATCGGG GCGAGGAATATCGGC AGGAGGAAGGCCGGG AGGAGACGTATCGGG AGGAGGATAGTCGGG AGGAGGAATAAACGG 
